In [1]:
import numpy as np
import pandas as pd

In [2]:
file_path = "/content/Cleaned_NYPD_Complaint_Data_Current__Year_To_Date_.csv"
df = pd.read_csv(file_path, index_col=0)

In [3]:
df.drop(["CMPLNT_NUM"], axis=1, inplace=True) # CMPLNT_NUM is just an identifier.

In [4]:
df

,ADDR_PCT_CD,BORO_NM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,CRM_ATPT_CPTD_CD,JURISDICTION_CODE,JURIS_DESC,KY_CD,LAW_CAT_CD,LOC_OF_OCCUR_DESC,OFNS_DESC,PATROL_BORO,PD_CD,PD_DESC,PREM_TYP_DESC,RPT_DT,SUSP_AGE_GROUP,SUSP_RACE,SUSP_SEX,VIC_AGE_GROUP,VIC_RACE,VIC_SEX,Latitude,Longitude,Lat_Lon,New Georeferenced Column
0,84,BROOKLYN,03/31/2021,15:08:00,03/31/2021,15:12:00,COMPLETED,0.0,N.Y. POLICE DEPT,675,VIOLATION,NaN,ADMINISTRATIVE CODE,PATROL BORO BKLYN NORTH,874.0,"PEDDLING,UNLAWFUL",STREET,03/31/2021,25-44,BLACK,M,UNKNOWN,UNKNOWN,E,40.691059,-73.986331,"(40.69105856700002, -73.98633121699999)",POINT (-73.98633121699999 40.69105856700002)
2,75,NaN,03/27/2021,13:00:00,NaN,NaN,COMPLETED,NaN,N.Y. POLICE DEPT,101,FELONY,OUTSIDE,MURDER & NON-NEGL. MANSLAUGHTER,NaN,NaN,NaN,NaN,03/27/2021,25-44,BLACK,M,25-44,BLACK,M,40.668188,-73.875137,"(40.66818803900002, -73.87513706799997)",POINT (-73.87513706799997 40.66818803900002)
3,14,MANHATTAN,03/26/2021,16:40:00,NaN,NaN,COMPLETED,0.0,N.Y. POLICE DEPT,341,MISDEMEANOR,INSIDE,PETIT LARCENY,PATROL BORO MAN SOUTH,333.0,"LARCENY,PETIT FROM STORE-SHOPL",DEPARTMENT STORE,03/26/2021,NaN,NaN,NaN,UNKNOWN,UNKNOWN,D,40.750431,-73.989282,"(40.75043076800005, -73.98928217599996)",POINT (-73.98928217599996 40.75043076800005)
4,110,QUEENS,03/26/2021,13:09:00,03/26/2021,14:29:00,COMPLETED,0.0,N.Y. POLICE DEPT,113,FELONY,INSIDE,FORGERY,PATROL BORO QUEENS NORTH,729.0,"FORGERY,ETC.,UNCLASSIFIED-FELO",SMALL MERCHANT,03/26/2021,NaN,NaN,NaN,UNKNOWN,UNKNOWN,D,40.749601,-73.865373,"(40.74960053300003, -73.86537333399998)",POINT (-73.86537333399998 40.74960053300003)
5,115,QUEENS,03/24/2021,18:06:00,03/24/2021,18:10:00,COMPLETED,0.0,N.Y. POLICE DEPT,347,MISDEMEANOR,NaN,INTOXICATED & IMPAIRED DRIVING,PATROL BORO QUEENS NORTH,905.0,"INTOXICATED DRIVING,ALCOHOL",STREET,03/24/2021,NaN,NaN,NaN,25-44,BLACK,F,40.762205,-73.870405,"(40.762204702000076, -73.870405222)",POINT (-73.870405222 40.762204702000076)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96729,78,BROOKLYN,01/03/2021,18:53:00,01/03/2021,19:33:00,COMPLETED,0.0,N.Y. POLICE DEPT,344,MISDEMEANOR,INSIDE,ASSAULT 3 & RELATED OFFENSES,PATROL BORO BKLYN SOUTH,101.0,ASSAULT 3,HOSPITAL,01/03/2021,<18,BLACK,F,45-64,WHITE HISPANIC,M,40.668147,-73.978896,"(40.668146816000046, -73.97889574699997)",POINT (-73.97889574699997 40.668146816000046)
96730,47,BRONX,01/02/2021,15:20:00,01/02/2021,15:25:00,COMPLETED,0.0,N.Y. POLICE DEPT,344,MISDEMEANOR,INSIDE,ASSAULT 3 & RELATED OFFENSES,PATROL BORO BRONX,101.0,ASSAULT 3,RESIDENCE - APT. HOUSE,01/02/2021,25-44,UNKNOWN,M,25-44,BLACK,F,40.892514,-73.859568,"(40.89251396600008, -73.859568127)",POINT (-73.859568127 40.89251396600008)
96731,40,BRONX,01/05/2021,12:00:00,01/06/2021,13:00:00,COMPLETED,0.0,N.Y. POLICE DEPT,341,MISDEMEANOR,INSIDE,PETIT LARCENY,PATROL BORO BRONX,313.0,"LARCENY,PETIT BY FALSE PROMISE",RESIDENCE - APT. HOUSE,01/06/2021,UNKNOWN,UNKNOWN,M,45-64,BLACK HISPANIC,M,40.819444,-73.908468,"(40.81944420600007, -73.90846796099999)",POINT (-73.90846796099999 40.81944420600007)
96732,14,MANHATTAN,01/04/2021,14:00:00,01/04/2021,14:05:00,COMPLETED,0.0,N.Y. POLICE DEPT,344,MISDEMEANOR,FRONT OF,ASSAULT 3 & RELATED OFFENSES,PATROL BORO MAN SOUTH,101.0,ASSAULT 3,STREET,01/04/2021,25-44,WHITE,M,45-64,BLACK,M,40.750022,-73.991339,"(40.75002197300005, -73.99133946299997)",POINT (-73.99133946299997 40.75002197300005)


In [5]:
# date_cols = ["CMPLNT_FR_DT", "CMPLNT_TO_DT", "RPT_DT"]
# pd.to_datetime(df[date_cols].stack(), format="%m-%d-%Y").unstack()

In [6]:
# Preparing data

# Combine date and time columns
orig_date_fr_cols = ["CMPLNT_FR_DT", "CMPLNT_FR_TM"]
df["CMPLNT_FR_DTM"] = df[orig_date_fr_cols].apply(lambda x: " ".join(x), axis=1)
df.drop(orig_date_fr_cols, axis=1, inplace=True)

"""
# def related_to_cols_TO(df, cols):
#   for col in cols:
#     if col == np.nan:
#       continue
#     else:
#       cols_ = " ".join(cols)
#   return 

# orig_date_to_cols = ["CMPLNT_TO_DT", "CMPLNT_TO_TM"]
# df["CMPLNT_TO_DTM"] = df[orig_date_to_cols].apply(related_to_cols_TO)
# df.drop(orig_date_to_cols, axis=1, inplace=False)

"""
# TODO: Problem with TO_ cols since they have NaN values
# orig_date_to_cols = ["CMPLNT_TO_DT", "CMPLNT_TO_TM"]
# df["CMPLNT_FR_DTM"] = df[orig_date_to_cols].apply(lambda x: " ".join(x), axis=1)
# df.drop(orig_date_to_cols, axis=1, inplace=True)

'\n# def related_to_cols_TO(df, cols):\n#   for col in cols:\n#     if col == np.nan:\n#       continue\n#     else:\n#       cols_ = " ".join(cols)\n#   return \n\n# orig_date_to_cols = ["CMPLNT_TO_DT", "CMPLNT_TO_TM"]\n# df["CMPLNT_TO_DTM"] = df[orig_date_to_cols].apply(related_to_cols_TO)\n# df.drop(orig_date_to_cols, axis=1, inplace=False)\n\n'

In [7]:
# categorical_features = ["BORO_NM", "CRM_ATPT_CPTD_CD"]
categorical_features = ["BORO_NM", "CRM_ATPT_CPTD_CD", "JURIS_DESC", "LOC_OF_OCCUR_DESC", 
                "OFNS_DESC", "PATROL_BORO", "PD_DESC", "PREM_TYP_DESC", "SUSP_RACE", "SUSP_SEX", 
                "VIC_AGE_GROUP", "VIC_RACE", "VIC_SEX"
              ]
categorical_targets = ["LAW_CAT_CD"]

df = pd.get_dummies(df, columns=categorical_features)

In [8]:
# One hot encoding the target variable
from sklearn.preprocessing import LabelEncoder
y = df["LAW_CAT_CD"]
le = LabelEncoder()
series = le.fit_transform(y)

df["Target"] = series
df.drop("LAW_CAT_CD", axis=1, inplace=True)

In [9]:
# Removing spurious age groups.
bad_susp_age_grps = pd.Series([np.nan, "UNKNOWN", "-947", "2021", "-973", "1017", "-966"])
df = df[~df["SUSP_AGE_GROUP"].isin(bad_susp_age_grps)]

In [10]:
df # `Target` is the one-hot encoded version of LAW_CT_CD

,ADDR_PCT_CD,CMPLNT_TO_DT,CMPLNT_TO_TM,JURISDICTION_CODE,KY_CD,PD_CD,RPT_DT,SUSP_AGE_GROUP,Latitude,Longitude,Lat_Lon,New Georeferenced Column,CMPLNT_FR_DTM,BORO_NM_BRONX,BORO_NM_BROOKLYN,BORO_NM_MANHATTAN,BORO_NM_QUEENS,BORO_NM_STATEN ISLAND,CRM_ATPT_CPTD_CD_ATTEMPTED,CRM_ATPT_CPTD_CD_COMPLETED,JURIS_DESC_AMTRACK,JURIS_DESC_DEPT OF CORRECTIONS,JURIS_DESC_HEALTH & HOSP CORP,JURIS_DESC_LONG ISLAND RAILRD,JURIS_DESC_METRO NORTH,JURIS_DESC_N.Y. HOUSING POLICE,JURIS_DESC_N.Y. POLICE DEPT,JURIS_DESC_N.Y. STATE POLICE,JURIS_DESC_N.Y. TRANSIT POLICE,JURIS_DESC_NEW YORK CITY SHERIFF OFFICE,JURIS_DESC_NYC PARKS,JURIS_DESC_NYS DEPT TAX AND FINANCE,JURIS_DESC_OTHER,JURIS_DESC_PORT AUTHORITY,JURIS_DESC_STATN IS RAPID TRANS,JURIS_DESC_TRI-BORO BRDG TUNNL,JURIS_DESC_U.S. PARK POLICE,LOC_OF_OCCUR_DESC_FRONT OF,LOC_OF_OCCUR_DESC_INSIDE,LOC_OF_OCCUR_DESC_OPPOSITE OF,...,PREM_TYP_DESC_TELECOMM. STORE,PREM_TYP_DESC_TRAMWAY,PREM_TYP_DESC_TRANSIT - NYC SUBWAY,PREM_TYP_DESC_TRANSIT FACILITY (OTHER),PREM_TYP_DESC_TUNNEL,PREM_TYP_DESC_VARIETY STORE,PREM_TYP_DESC_VIDEO STORE,SUSP_RACE_AMERICAN INDIAN/ALASKAN NATIVE,SUSP_RACE_ASIAN / PACIFIC ISLANDER,SUSP_RACE_BLACK,SUSP_RACE_BLACK HISPANIC,SUSP_RACE_UNKNOWN,SUSP_RACE_WHITE,SUSP_RACE_WHITE HISPANIC,SUSP_SEX_F,SUSP_SEX_M,SUSP_SEX_U,VIC_AGE_GROUP_-1,VIC_AGE_GROUP_-48,VIC_AGE_GROUP_-51,VIC_AGE_GROUP_-935,VIC_AGE_GROUP_18-24,VIC_AGE_GROUP_25-44,VIC_AGE_GROUP_45-64,VIC_AGE_GROUP_65+,VIC_AGE_GROUP_<18,VIC_AGE_GROUP_UNKNOWN,VIC_RACE_AMERICAN INDIAN/ALASKAN NATIVE,VIC_RACE_ASIAN / PACIFIC ISLANDER,VIC_RACE_BLACK,VIC_RACE_BLACK HISPANIC,VIC_RACE_UNKNOWN,VIC_RACE_WHITE,VIC_RACE_WHITE HISPANIC,VIC_SEX_D,VIC_SEX_E,VIC_SEX_F,VIC_SEX_M,VIC_SEX_U,Target
0,84,03/31/2021,15:12:00,0.0,675,874.0,03/31/2021,25-44,40.691059,-73.986331,"(40.69105856700002, -73.98633121699999)",POINT (-73.98633121699999 40.69105856700002),03/31/2021 15:08:00,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,2
2,75,NaN,NaN,NaN,101,NaN,03/27/2021,25-44,40.668188,-73.875137,"(40.66818803900002, -73.87513706799997)",POINT (-73.87513706799997 40.66818803900002),03/27/2021 13:00:00,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0
8,43,NaN,NaN,0.0,107,221.0,03/22/2021,25-44,40.824941,-73.878322,"(40.82494145800007, -73.87832247299998)",POINT (-73.87832247299998 40.82494145800007),03/22/2021 17:45:00,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0
12,47,NaN,NaN,0.0,113,729.0,03/19/2021,25-44,40.899513,-73.857000,"(40.899512505000075, -73.85699953999993)",POINT (-73.85699953999993 40.899512505000075),03/19/2021 21:00:00,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0
13,103,NaN,NaN,NaN,101,NaN,03/19/2021,25-44,40.701744,-73.814592,"(40.70174365200006, -73.81459233099997)",POINT (-73.81459233099997 40.70174365200006),03/19/2021 01:28:00,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96724,101,01/04/2021,17:25:00,0.0,107,221.0,01/04/2021,45-64,40.598256,-73.755230,"(40.59825555300005, -73.75522965799998)",POINT (-73.75522965799998 40.59825555300005),01/04/2021 17:20:00,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0
96728,63,01/05/2021,07:25:00,0.0,344,113.0,01/05/2021,25-44,40.615744,-73.915580,"(40.6157

In [11]:
df["SUSP_AGE_GROUP"].unique()

array(['25-44', '45-64', '18-24', '65+', '<18'], dtype=object)

In [12]:
df = pd.get_dummies(df, columns=["SUSP_AGE_GROUP"])

In [13]:
df

,ADDR_PCT_CD,CMPLNT_TO_DT,CMPLNT_TO_TM,JURISDICTION_CODE,KY_CD,PD_CD,RPT_DT,Latitude,Longitude,Lat_Lon,New Georeferenced Column,CMPLNT_FR_DTM,BORO_NM_BRONX,BORO_NM_BROOKLYN,BORO_NM_MANHATTAN,BORO_NM_QUEENS,BORO_NM_STATEN ISLAND,CRM_ATPT_CPTD_CD_ATTEMPTED,CRM_ATPT_CPTD_CD_COMPLETED,JURIS_DESC_AMTRACK,JURIS_DESC_DEPT OF CORRECTIONS,JURIS_DESC_HEALTH & HOSP CORP,JURIS_DESC_LONG ISLAND RAILRD,JURIS_DESC_METRO NORTH,JURIS_DESC_N.Y. HOUSING POLICE,JURIS_DESC_N.Y. POLICE DEPT,JURIS_DESC_N.Y. STATE POLICE,JURIS_DESC_N.Y. TRANSIT POLICE,JURIS_DESC_NEW YORK CITY SHERIFF OFFICE,JURIS_DESC_NYC PARKS,JURIS_DESC_NYS DEPT TAX AND FINANCE,JURIS_DESC_OTHER,JURIS_DESC_PORT AUTHORITY,JURIS_DESC_STATN IS RAPID TRANS,JURIS_DESC_TRI-BORO BRDG TUNNL,JURIS_DESC_U.S. PARK POLICE,LOC_OF_OCCUR_DESC_FRONT OF,LOC_OF_OCCUR_DESC_INSIDE,LOC_OF_OCCUR_DESC_OPPOSITE OF,LOC_OF_OCCUR_DESC_OUTSIDE,...,PREM_TYP_DESC_VARIETY STORE,PREM_TYP_DESC_VIDEO STORE,SUSP_RACE_AMERICAN INDIAN/ALASKAN NATIVE,SUSP_RACE_ASIAN / PACIFIC ISLANDER,SUSP_RACE_BLACK,SUSP_RACE_BLACK HISPANIC,SUSP_RACE_UNKNOWN,SUSP_RACE_WHITE,SUSP_RACE_WHITE HISPANIC,SUSP_SEX_F,SUSP_SEX_M,SUSP_SEX_U,VIC_AGE_GROUP_-1,VIC_AGE_GROUP_-48,VIC_AGE_GROUP_-51,VIC_AGE_GROUP_-935,VIC_AGE_GROUP_18-24,VIC_AGE_GROUP_25-44,VIC_AGE_GROUP_45-64,VIC_AGE_GROUP_65+,VIC_AGE_GROUP_<18,VIC_AGE_GROUP_UNKNOWN,VIC_RACE_AMERICAN INDIAN/ALASKAN NATIVE,VIC_RACE_ASIAN / PACIFIC ISLANDER,VIC_RACE_BLACK,VIC_RACE_BLACK HISPANIC,VIC_RACE_UNKNOWN,VIC_RACE_WHITE,VIC_RACE_WHITE HISPANIC,VIC_SEX_D,VIC_SEX_E,VIC_SEX_F,VIC_SEX_M,VIC_SEX_U,Target,SUSP_AGE_GROUP_18-24,SUSP_AGE_GROUP_25-44,SUSP_AGE_GROUP_45-64,SUSP_AGE_GROUP_65+,SUSP_AGE_GROUP_<18
0,84,03/31/2021,15:12:00,0.0,675,874.0,03/31/2021,40.691059,-73.986331,"(40.69105856700002, -73.98633121699999)",POINT (-73.98633121699999 40.69105856700002),03/31/2021 15:08:00,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,2,0,1,0,0,0
2,75,NaN,NaN,NaN,101,NaN,03/27/2021,40.668188,-73.875137,"(40.66818803900002, -73.87513706799997)",POINT (-73.87513706799997 40.66818803900002),03/27/2021 13:00:00,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0
8,43,NaN,NaN,0.0,107,221.0,03/22/2021,40.824941,-73.878322,"(40.82494145800007, -73.87832247299998)",POINT (-73.87832247299998 40.82494145800007),03/22/2021 17:45:00,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0
12,47,NaN,NaN,0.0,113,729.0,03/19/2021,40.899513,-73.857000,"(40.899512505000075, -73.85699953999993)",POINT (-73.85699953999993 40.899512505000075),03/19/2021 21:00:00,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0
13,103,NaN,NaN,NaN,101,NaN,03/19/2021,40.701744,-73.814592,"(40.70174365200006, -73.81459233099997)",POINT (-73.81459233099997 40.70174365200006),03/19/2021 01:28:00,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96724,101,01/04/2021,17:25:00,0.0,107,221.0,01/04/2021,40.598256,-73.755230,"(40.59825555300005, -73.75522965799998)",POINT (-73.75522965799998 40.59825555300005),01/04/2021 17:20:00,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0
96728,63,01/05/2021,07:25:00,0.0,344,113.0,01/05/2021,40.615744,-73.915580,"(40.61574449600005, -73.91558008099996)",POINT (-73.91558008099996 40.615

In [14]:
date_cols = ["CMPLNT_TO_DT", "CMPLNT_TO_TM", "RPT_DT", "CMPLNT_FR_DTM"]
new_df = df.drop(date_cols, axis=1)

# Temporary
additional_drop_cols = ["Lat_Lon", "New Georeferenced Column"]
new_df.drop(additional_drop_cols, axis=1, inplace=True)

# Remove this drop
check_drop_cols = ["JURISDICTION_CODE", "PD_CD"]
new_df.drop(check_drop_cols, axis=1, inplace=True)

In [15]:
new_df

,ADDR_PCT_CD,KY_CD,Latitude,Longitude,BORO_NM_BRONX,BORO_NM_BROOKLYN,BORO_NM_MANHATTAN,BORO_NM_QUEENS,BORO_NM_STATEN ISLAND,CRM_ATPT_CPTD_CD_ATTEMPTED,CRM_ATPT_CPTD_CD_COMPLETED,JURIS_DESC_AMTRACK,JURIS_DESC_DEPT OF CORRECTIONS,JURIS_DESC_HEALTH & HOSP CORP,JURIS_DESC_LONG ISLAND RAILRD,JURIS_DESC_METRO NORTH,JURIS_DESC_N.Y. HOUSING POLICE,JURIS_DESC_N.Y. POLICE DEPT,JURIS_DESC_N.Y. STATE POLICE,JURIS_DESC_N.Y. TRANSIT POLICE,JURIS_DESC_NEW YORK CITY SHERIFF OFFICE,JURIS_DESC_NYC PARKS,JURIS_DESC_NYS DEPT TAX AND FINANCE,JURIS_DESC_OTHER,JURIS_DESC_PORT AUTHORITY,JURIS_DESC_STATN IS RAPID TRANS,JURIS_DESC_TRI-BORO BRDG TUNNL,JURIS_DESC_U.S. PARK POLICE,LOC_OF_OCCUR_DESC_FRONT OF,LOC_OF_OCCUR_DESC_INSIDE,LOC_OF_OCCUR_DESC_OPPOSITE OF,LOC_OF_OCCUR_DESC_OUTSIDE,LOC_OF_OCCUR_DESC_REAR OF,OFNS_DESC_ADMINISTRATIVE CODE,OFNS_DESC_AGRICULTURE & MRKTS LAW-UNCLASSIFIED,OFNS_DESC_ALCOHOLIC BEVERAGE CONTROL LAW,OFNS_DESC_ANTICIPATORY OFFENSES,OFNS_DESC_ARSON,OFNS_DESC_ASSAULT 3 & RELATED OFFENSES,OFNS_DESC_BURGLAR'S TOOLS,...,PREM_TYP_DESC_VARIETY STORE,PREM_TYP_DESC_VIDEO STORE,SUSP_RACE_AMERICAN INDIAN/ALASKAN NATIVE,SUSP_RACE_ASIAN / PACIFIC ISLANDER,SUSP_RACE_BLACK,SUSP_RACE_BLACK HISPANIC,SUSP_RACE_UNKNOWN,SUSP_RACE_WHITE,SUSP_RACE_WHITE HISPANIC,SUSP_SEX_F,SUSP_SEX_M,SUSP_SEX_U,VIC_AGE_GROUP_-1,VIC_AGE_GROUP_-48,VIC_AGE_GROUP_-51,VIC_AGE_GROUP_-935,VIC_AGE_GROUP_18-24,VIC_AGE_GROUP_25-44,VIC_AGE_GROUP_45-64,VIC_AGE_GROUP_65+,VIC_AGE_GROUP_<18,VIC_AGE_GROUP_UNKNOWN,VIC_RACE_AMERICAN INDIAN/ALASKAN NATIVE,VIC_RACE_ASIAN / PACIFIC ISLANDER,VIC_RACE_BLACK,VIC_RACE_BLACK HISPANIC,VIC_RACE_UNKNOWN,VIC_RACE_WHITE,VIC_RACE_WHITE HISPANIC,VIC_SEX_D,VIC_SEX_E,VIC_SEX_F,VIC_SEX_M,VIC_SEX_U,Target,SUSP_AGE_GROUP_18-24,SUSP_AGE_GROUP_25-44,SUSP_AGE_GROUP_45-64,SUSP_AGE_GROUP_65+,SUSP_AGE_GROUP_<18
0,84,675,40.691059,-73.986331,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,2,0,1,0,0,0
2,75,101,40.668188,-73.875137,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0
8,43,107,40.824941,-73.878322,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0
12,47,113,40.899513,-73.857000,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0
13,103,101,40.701744,-73.814592,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96724,101,107,40.598256,-73.755230,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0
96728,63,344,40.615744,-73.915580,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0
96729,78,344,40.668147,-73.978896,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,1
96730,47,344,40.892514,-73.859568,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0


In [16]:
# def cat_targets_to_one_hot(df, cat_targets):
#   for cat_target in cat_targets:
#     unique_values = df[cat_target].unique()
#     for unique_value in unique_value:
#       df[str(cat_target)+str(unique_value)] = 

In [17]:
# categorical = ["BORO_NM", "CRM_ATPT_CPTD_CD", "JURIS_DESC", "LAW_CAT_CD", "LOC_OF_OCCUR_DESC", "OFNS_DESC", "PATROL_BORO", "PD_DESC", "PREM_TYP_DESC", "SUSP_RACE", "SUSP_SEX", "VIC_AGE_GROUP", "VIC_RACE", "VIC_SEX"]
# final_data = pd.get_dummies(df, columns=categorical)

In [18]:
from sklearn.model_selection import train_test_split

In [19]:
# Needed for classifiers like the XGBoost
new_df = new_df.rename(columns={"SUSP_AGE_GROUP_<18": "SUSP_AGE_GROUP_less_than_18"})

In [20]:
X = new_df.drop("Target", axis=1)
y = new_df["Target"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

In [39]:
X

,ADDR_PCT_CD,KY_CD,Latitude,Longitude,BORO_NM_BRONX,BORO_NM_BROOKLYN,BORO_NM_MANHATTAN,BORO_NM_QUEENS,BORO_NM_STATEN ISLAND,CRM_ATPT_CPTD_CD_ATTEMPTED,CRM_ATPT_CPTD_CD_COMPLETED,JURIS_DESC_AMTRACK,JURIS_DESC_DEPT OF CORRECTIONS,JURIS_DESC_HEALTH & HOSP CORP,JURIS_DESC_LONG ISLAND RAILRD,JURIS_DESC_METRO NORTH,JURIS_DESC_N.Y. HOUSING POLICE,JURIS_DESC_N.Y. POLICE DEPT,JURIS_DESC_N.Y. STATE POLICE,JURIS_DESC_N.Y. TRANSIT POLICE,JURIS_DESC_NEW YORK CITY SHERIFF OFFICE,JURIS_DESC_NYC PARKS,JURIS_DESC_NYS DEPT TAX AND FINANCE,JURIS_DESC_OTHER,JURIS_DESC_PORT AUTHORITY,JURIS_DESC_STATN IS RAPID TRANS,JURIS_DESC_TRI-BORO BRDG TUNNL,JURIS_DESC_U.S. PARK POLICE,LOC_OF_OCCUR_DESC_FRONT OF,LOC_OF_OCCUR_DESC_INSIDE,LOC_OF_OCCUR_DESC_OPPOSITE OF,LOC_OF_OCCUR_DESC_OUTSIDE,LOC_OF_OCCUR_DESC_REAR OF,OFNS_DESC_ADMINISTRATIVE CODE,OFNS_DESC_AGRICULTURE & MRKTS LAW-UNCLASSIFIED,OFNS_DESC_ALCOHOLIC BEVERAGE CONTROL LAW,OFNS_DESC_ANTICIPATORY OFFENSES,OFNS_DESC_ARSON,OFNS_DESC_ASSAULT 3 & RELATED OFFENSES,OFNS_DESC_BURGLAR'S TOOLS,...,PREM_TYP_DESC_TUNNEL,PREM_TYP_DESC_VARIETY STORE,PREM_TYP_DESC_VIDEO STORE,SUSP_RACE_AMERICAN INDIAN/ALASKAN NATIVE,SUSP_RACE_ASIAN / PACIFIC ISLANDER,SUSP_RACE_BLACK,SUSP_RACE_BLACK HISPANIC,SUSP_RACE_UNKNOWN,SUSP_RACE_WHITE,SUSP_RACE_WHITE HISPANIC,SUSP_SEX_F,SUSP_SEX_M,SUSP_SEX_U,VIC_AGE_GROUP_-1,VIC_AGE_GROUP_-48,VIC_AGE_GROUP_-51,VIC_AGE_GROUP_-935,VIC_AGE_GROUP_18-24,VIC_AGE_GROUP_25-44,VIC_AGE_GROUP_45-64,VIC_AGE_GROUP_65+,VIC_AGE_GROUP_<18,VIC_AGE_GROUP_UNKNOWN,VIC_RACE_AMERICAN INDIAN/ALASKAN NATIVE,VIC_RACE_ASIAN / PACIFIC ISLANDER,VIC_RACE_BLACK,VIC_RACE_BLACK HISPANIC,VIC_RACE_UNKNOWN,VIC_RACE_WHITE,VIC_RACE_WHITE HISPANIC,VIC_SEX_D,VIC_SEX_E,VIC_SEX_F,VIC_SEX_M,VIC_SEX_U,SUSP_AGE_GROUP_18-24,SUSP_AGE_GROUP_25-44,SUSP_AGE_GROUP_45-64,SUSP_AGE_GROUP_65+,SUSP_AGE_GROUP_less_than_18
0,84,675,40.691059,-73.986331,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0
2,75,101,40.668188,-73.875137,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0
8,43,107,40.824941,-73.878322,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0
12,47,113,40.899513,-73.857000,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0
13,103,101,40.701744,-73.814592,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96724,101,107,40.598256,-73.755230,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0
96728,63,344,40.615744,-73.915580,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0
96729,78,344,40.668147,-73.978896,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1
96730,47,344,40.892514,-73.859568,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0


In [21]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.cluster import KMeans
from sklearn.svm import SVC

In [22]:
dtree = DecisionTreeClassifier()
kmeans = KMeans(n_clusters=3)
svc = SVC()

In [23]:
kmeans.fit(X_train, y_train)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=3, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [24]:
predictions = kmeans.predict(X_test)

In [25]:
print(f"Accuracy: {np.sum(predictions == y_test) / len(y_test)}")

Accuracy: 0.22382449950141903


In [26]:
avg_acc = 0
iters = 50
for _ in range(iters):
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)
  kmeans = KMeans(n_clusters=3)
  kmeans.fit(X_train, y_train)
  predictions = kmeans.predict(X_test)
  acc = np.sum(predictions == y_test) / len(y_test)
  print(f"Accuracy: {acc}")
  avg_acc += acc
print(f"Average Accuracy: {avg_acc/iters}")

Accuracy: 1.0
Accuracy: 0.290711053156401
Accuracy: 0.22382449950141903
Accuracy: 0.22382449950141903
Accuracy: 0.22382449950141903
Accuracy: 1.0
Accuracy: 1.0
Accuracy: 0.22382449950141903
Accuracy: 0.22382449950141903
Accuracy: 0.0
Accuracy: 0.22382449950141903
Accuracy: 1.0
Accuracy: 0.48546444734217997
Accuracy: 0.0
Accuracy: 0.22382449950141903
Accuracy: 1.0
Accuracy: 1.0
Accuracy: 1.0
Accuracy: 0.22382449950141903
Accuracy: 0.22382449950141903
Accuracy: 0.0
Accuracy: 0.0
Accuracy: 0.22382449950141903
Accuracy: 0.48546444734217997
Accuracy: 1.0
Accuracy: 1.0
Accuracy: 0.0
Accuracy: 0.22382449950141903
Accuracy: 0.0
Accuracy: 0.290711053156401
Accuracy: 0.22382449950141903
Accuracy: 0.22382449950141903
Accuracy: 0.22382449950141903
Accuracy: 1.0
Accuracy: 1.0
Accuracy: 1.0
Accuracy: 0.0
Accuracy: 0.22382449950141903
Accuracy: 1.0
Accuracy: 0.22382449950141903
Accuracy: 1.0
Accuracy: 0.290711053156401
Accuracy: 1.0
Accuracy: 0.22382449950141903
Accuracy: 0.48546444734217997
Accuracy

In [27]:
from sklearn.ensemble import GradientBoostingClassifier

In [28]:
gbc = GradientBoostingClassifier(max_depth=10, learning_rate=0.01)

In [29]:
gbc.fit(X_train, y_train)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.01, loss='deviance', max_depth=10,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [30]:
gbc_preds = gbc.predict(X_test)

In [31]:
print(f"GBC Accuracy: {np.sum(gbc_preds == y_test) / len(y_test)}")

GBC Accuracy: 1.0


In [32]:
import xgboost as xgb

In [33]:
xgb_classifier = xgb.XGBClassifier(n_estimators=3)

In [ ]:
# xgb_classifier.fit(X_train, y_train) DOESNT WORK NOW..